In [ ]:
import copy
import datetime
import ijson
import IPython.display
import tabulate


def to_datetime(str):
    return datetime.datetime.strptime(str, "%Y-%m-%dT%H:%M:%S.%f")


def to_date(when):
    return when.strftime("%Y-%m-%d")


def to_time(when):
    return when.strftime("%H:%M:%S.%f")


def to_price(str):
    split = str.split(".")
    if len(split) != 1 and int(split[1]) != 0:
        raise Exception("Fractional prices not supported!")
    return int(split[0])


def find_trade(connection, when, product, quantity, price):
    start = to_datetime(when)
    end = start + datetime.timedelta(microseconds=1)
    query = {
        "query": "ticks",
        "messages": ["TRD"],
        "source": "cme",
        "product": product,
        "start_date": to_date(start),
        "end_date": to_date(end),
        "start_time": to_time(start),
        "end_time": to_time(end),
        "time_zone": "UTC",
        "timestamp_mode": "exchange",
    }

    def predicate(obj):
        return (
            obj["printable"]
            and obj["quantity"] == int(quantity)
            and obj["price"] == to_price(price)
        )

    for trade in filter(predicate, ijson.items(connection.stream(query), "item")):
        connection.cancel()
        return trade
    return None


def split_timestamp(when):
    ns = when % 1000000000
    s = when / 1000000000
    dt = datetime.datetime.utcfromtimestamp(s)
    ns_str = str(ns)
    while len(ns_str) != 9:
        ns_str = "0" + ns_str
    return (to_date(dt), dt.strftime("%H:%M:%S.") + ns_str)


def split_now():
    now = datetime.datetime.now()
    s = int(now.timestamp())
    return split_timestamp(s * 1000000000)


def snapshot(connection, when, product):
    (date, time) = split_timestamp(when)
    query = {
        "query": "snapshot",
        "type": "levels",
        "source": "cme",
        "product": product,
        "date": date,
        "time": time,
        "timestamp_mode": "exchange",
        "time_zone": "UTC",
    }

    def format(obj):
        obj = copy.copy(obj)
        del obj["product_id"]
        del obj["name"]
        del obj["type"]
        del obj["snapshot"]
        del obj["buy"]
        del obj["implied"]
        del obj["message_number"]
        del obj["transaction_id"]
        return hpq.format(obj)

    arr = list(ijson.items(connection.stream(query), "item"))

    def table(pred):
        local = list(map(format, filter(pred, arr)))
        if len(local) == 0:
            IPython.display.display(IPython.display.HTML("<p>Empty</p>"))
        else:
            table = tabulate.tabulate(local, tablefmt="html", headers="keys")
            IPython.display.display(IPython.display.HTML(table))

    IPython.display.display(IPython.display.HTML("<h3>Outright Bid</h3>"))
    table(lambda obj: obj["buy"] and not obj["implied"])
    IPython.display.display(IPython.display.HTML("<h3>Outright Ask</h3>"))
    table(lambda obj: not obj["buy"] and not obj["implied"])
    IPython.display.display(IPython.display.HTML("<h3>Implied Bid</h3>"))
    table(lambda obj: obj["buy"] and obj["implied"])
    IPython.display.display(IPython.display.HTML("<h3>Implied Ask</h3>"))
    table(lambda obj: not obj["buy"] and obj["implied"])


def next_top_of_book(connection, when, product):
    (start_date, start_time) = split_timestamp(when)
    (end_date, end_time) = split_now()
    query = {
        "query": "topofbook",
        "source": "cme",
        "product": product,
        "start_date": start_date,
        "start_time": start_time,
        "end_date": end_date,
        "end_time": end_time,
        "timestamp_mode": "exchange",
        "time_zone": "UTC",
    }
    for obj in ijson.items(connection.stream(query), "item"):
        connection.cancel()
        del obj["type"]
        del obj["product_id"]
        del obj["name"]
        del obj["message_number"]
        del obj["transaction_id"]

        def flatten(key):
            inner = obj[key]
            del obj[key]
            if inner is not None:
                del inner["buy"]
                del inner["level"]
                del inner["message_number"]
                del inner["transaction_id"]
                for k in inner.keys():
                    actual = inner[k]
                    if k == "exchange_timestamp" or k == "receipt_timestamp":
                        actual = hpq.format_timestamp(actual)
                    obj[key + "_" + k] = actual

        if "bid" in obj.keys():
            flatten("bid")
        if "ask" in obj.keys():
            flatten("ask")
        obj = hpq.format(obj)
        table = tabulate.tabulate([obj], tablefmt="html", headers="keys")
        IPython.display.display(IPython.display.HTML(table))
        return
    IPython.display.display(IPython.display.HTML("<p>None</p>"))


def next_trade(connection, when, product):
    (start_date, start_time) = split_timestamp(when)
    (end_date, end_time) = split_now()
    query = {
        "query": "ticks",
        "source": "cme",
        "product": product,
        "start_date": start_date,
        "start_time": start_time,
        "end_date": end_date,
        "end_time": end_time,
        "timestamp_mode": "exchange",
        "time_zone": "UTC",
        "messages": ["TRD"],
    }

    def predicate(obj):
        return obj["printable"]

    for obj in filter(predicate, ijson.items(connection.stream(query), "item")):
        connection.cancel()
        del obj["type"]
        del obj["product_id"]
        del obj["name"]
        del obj["message_number"]
        del obj["transaction_id"]
        del obj["printable"]
        obj = hpq.format(obj)
        table = tabulate.tabulate([obj], tablefmt="html", headers="keys")
        IPython.display.display(IPython.display.HTML(table))
        return
    IPython.display.display(IPython.display.HTML("<p>None</p>"))

In [ ]:
raw = """ReceiptTimestamp,ExchangeTimestamp,Product,Quantity,Price
2022-10-27T05:04:41.013245,2022-10-27T05:04:41.012978,ESZ2,2,386200.0
2022-10-27T05:04:52.244588,2022-10-27T05:04:52.243857,ESZ2,1,386175.0
2022-10-27T05:04:58.258967,2022-10-27T05:04:58.258742,ESZ2,1,386150.0
2022-10-27T05:05:05.264367,2022-10-27T05:05:05.264131,ESZ2,3,386200.0
2022-10-27T05:05:20.574579,2022-10-27T05:05:20.574286,ESZ2,9,386150.0"""

In [ ]:
import csv
import hpq
import html
import IPython.display
import io

reader = csv.reader(io.StringIO(raw), delimiter=",")
connection = hpq.create_web_socket_client()
for trade in hpq.skip(1, reader):
    when = to_datetime(trade[1])
    when_formatted = when.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
    product = html.escape(trade[2])
    quantity = html.escape(trade[3])
    price = html.escape(trade[4])
    IPython.display.display(
        IPython.display.HTML(
            f"<h1>{when_formatted}: {product} {quantity} @ {price}</h1>"
        )
    )
    actual = find_trade(connection, *trade[1:5])
    if actual is None:
        IPython.display.display(IPython.display.HTML("<p>Not found</p>"))
        continue
    exchange_timestamp = actual["exchange_timestamp"]
    exchange_timestamp_formatted = hpq.format_timestamp(exchange_timestamp)
    IPython.display.display(
        IPython.display.HTML(
            f"<h2>Snapshot Preceding Trade ({exchange_timestamp_formatted})</h2>"
        )
    )
    snapshot(connection, exchange_timestamp, trade[2])
    next_exchange_timestamp = exchange_timestamp + 1
    next_exchange_timestamp_formatted = hpq.format_timestamp(next_exchange_timestamp)
    IPython.display.display(
        IPython.display.HTML(
            f"<h2>Snapshot After Trade ({next_exchange_timestamp_formatted})</h2>"
        )
    )
    snapshot(connection, next_exchange_timestamp, trade[2])
    IPython.display.display(IPython.display.HTML("<h2>Next Top of Book Change</h2>"))
    next_top_of_book(connection, next_exchange_timestamp, trade[2])
    IPython.display.display(IPython.display.HTML("<h2>Next Trade</h2>"))
    next_trade(connection, next_exchange_timestamp, trade[2])